In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
import json
import requests
from bs4 import BeautifulSoup

In [2]:
#Setting up the webdriver
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
global driver
#options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

In [3]:
#Signing into the account, and going to the needed URL
driver.get("https://www.bgvgrandcentral.com/")
username = driver.find_element_by_name("loginModel.Username").send_keys("")
Password = driver.find_element_by_name("loginModel.Password").send_keys("")
form = driver.find_element_by_tag_name("form")
form.submit()
driver.get("https://www.bgvgrandcentral.com/reservations/bonus-time")
driver.implicitly_wait(15)
classes = driver.find_element_by_class_name("container")
path_GC8 = '//*[@id="tabGC8"]/a'
path_GL7 = '//*[@id="tabGL7"]/a'
path_GTL = '//*[@id="tabGTL"]/a'
path_GP = '//*[@id="tabGP"]/a'
#list_1 is webelements associated to available dates
list_1 = []
#list_3 refreshed list of webelements after one is clicked on
list_3 = []
myDict = {}

In [4]:
#Selects the lodge via xpath, collects avaialable dates, runs procedure-calls the whole program
def select_lodge(lodge):
  Calendar = []
  myDate = ''
  driver.get("https://www.bgvgrandcentral.com/reservations/bonus-time")
  driver.implicitly_wait(5)
  try:
    driver.implicitly_wait(15)
    driver.find_element_by_xpath(lodge).click()
  except:
    driver.implicitly_wait(30)
    driver.find_element_by_xpath(lodge).click()
  #print(lodge)
  month_dates = get_dates()
  get_date_webelements()
  lodgename = get_lodge_name(lodge)
  print(lodgename)
  print("month_dates:",month_dates)
  Procedure(month_dates,lodge,lodgename)

In [5]:
#Creates a list of the month dates available
def get_dates():
  month_dates = []
  Calendar = driver.find_elements_by_class_name("selectable")
  Date = 0
  for myDate in Calendar: 
    try:
        Calendar = driver.find_elements_by_class_name("selectable")
        D = (myDate.find_element_by_tag_name("a"))
        Date = int(D.text)
        month_dates.extend([Date])
        driver.implicitly_wait(3)
    except:
        pass
  return month_dates

#Collects the webelements connected to the month dates available
def get_date_webelements(): 
  list_1 = []
  Calendar = driver.find_elements_by_class_name("selectable")
  for myDate in Calendar:     
    try:
        Calendar = driver.find_elements_by_class_name("selectable")
        D = (myDate.find_element_by_tag_name("a"))
        list_1.extend([D])
        driver.implicitly_wait(3)
    except:
        pass
  return list_1

#Supplies the lodge name to the dictKey in the dictionary
def get_lodge_name(lodge):
    if lodge == '//*[@id="tabGC8"]/a':
      lodgename = "GC8"
    elif lodge == '//*[@id="tabGL7"]/a':
      lodgename = "GL7"
    elif lodge == '//*[@id="tabGTL"]/a':
      lodgename = "GTL"
    else:
      lodgename = "GP"
    return lodgename

#refreshes the page to deselect previous dates
def refresh(lodge):
    homepage = driver.find_element_by_xpath('//*[@id="topNav"]/div/a/img').click()
    driver.implicitly_wait(10)
    driver.get("https://www.bgvgrandcentral.com/reservations/bonus-time")
    driver.implicitly_wait(5)
    try:
      driver.implicitly_wait(15)
      driver.find_element_by_xpath(lodge).click()
    except:
      driver.implicitly_wait(30)
      driver.find_element_by_xpath(lodge).click()
    driver.implicitly_wait(10)

In [6]:
#Calls the functions to create the dictionary of available dates
def Procedure (month_dates,lodge,lodgename):
  checkout = 0
  indexnumber = 0
  checkin = 0
  list_3 = []
  for Date in month_dates:
    indexnumber = month_dates.index(Date)
    checkout = Date+1
    checkin = str(Date)
    dictKey = (lodgename+"_"+checkin)
    if checkout in month_dates and indexnumber == 0:
      checkinout(indexnumber,Date,dictKey)
      
    elif checkout in month_dates:
      deselect_checkout(indexnumber,Date,lodge,dictKey)
    else:
      checkin_fill(indexnumber,Date,lodge,dictKey)
      

In [7]:
#Selects intial checkin/out dates
def checkinout (indexnumber,Date,dictKey):
    list_1 = get_date_webelements()
    list_1[indexnumber].click()
    driver.implicitly_wait(3)
    list_3 = (driver.find_elements_by_class_name("selectable"))
    list_3[indexnumber].click() 
    driver.implicitly_wait(3)
    form = driver.find_element_by_tag_name("form")
    form.submit()
    table(myDict,dictKey)
    
#Calls the refresh function and runs checkinout
def deselect_checkout(indexnumber,Date,lodge,dictKey):
    refresh(lodge)
    checkinout(indexnumber,Date,dictKey)

#Selects a single date that auto fills the checkout date          
def checkin_fill(indexnumber,Date,lodge,dictKey):
    refresh(lodge)
    list_1 = get_date_webelements()
    list_1[indexnumber].click()
    driver.implicitly_wait(3)
    form = driver.find_element_by_tag_name("form")
    form.submit()
    driver.implicitly_wait(3)
    table(myDict,dictKey)
    refresh(lodge)

In [8]:
#Populates the dictionary
def table(myDict, dictKey):
    driver.implicitly_wait(3)
    Table = driver.find_elements_by_xpath("/html/body/div[1]/div[5]/div/div[1]/div[2]/div[6]/div/table//tr/td")
    counter = 1
    myDict[dictKey] = ""
    for element in Table:
        myDict[dictKey] = myDict[dictKey] + ":" +(element.text)
    return myDict      

In [9]:
select_lodge(path_GC8)
select_lodge(path_GL7)
select_lodge(path_GTL)
#converts the dictionary to jsonselect_lodge(path_GP)
json_object = json.dumps(myDict, indent = 4)   
print(json_object)

GC8
month_dates: [28, 29, 30, 1, 2, 3, 4, 5, 9, 11]
GL7
month_dates: [28, 3]
GTL
month_dates: [28, 29, 30, 1, 2, 3, 4, 5]
{
    "GC8_28": ":Room Type:Rate\n(Includes tax & resort fees):Sleeps::Suite:111.88:4:Reserve:1 Bedroom Breckenridge:137.02:4:Reserve",
    "GC8_29": ":Room Type:Rate\n(Includes tax & resort fees):Sleeps::Suite:111.88:4:Reserve:1 Bedroom Breckenridge:137.02:4:Reserve",
    "GC8_30": ":Room Type:Rate\n(Includes tax & resort fees):Sleeps::Suite:137.02:4:Reserve",
    "GC8_1": ":Room Type:Rate\n(Includes tax & resort fees):Sleeps::Suite:111.88:4:Reserve:1 Bedroom Breckenridge:137.02:4:Reserve",
    "GC8_2": ":Room Type:Rate\n(Includes tax & resort fees):Sleeps::Suite:111.88:4:Reserve",
    "GC8_3": ":Room Type:Rate\n(Includes tax & resort fees):Sleeps::Suite:111.88:4:Reserve",
    "GC8_4": ":Room Type:Rate\n(Includes tax & resort fees):Sleeps::Suite:111.88:4:Reserve",
    "GC8_5": ":Room Type:Rate\n(Includes tax & resort fees):Sleeps::Suite:111.88:4:Reserve",
    "GC8_